In [2]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer, KNNImputer, MissingIndicator
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder, MinMaxScaler, PowerTransformer, OrdinalEncoder
from sklearn.model_selection import train_test_split

In [3]:
pip install dagshub mlflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.1/260.1 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.0/29.0 MB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 85.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 76.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 692.3/692.3 kB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
import dagshub
dagshub.init(repo_owner='speedyskill', repo_name='swiggy-delivery-time-prediction', mlflow=True)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

Output()



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=98e700b3-bbca-47a8-b307-f4c2ab658953&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=53d4d996fad4b9c58d90c8c3e5d688ed20c2b88b63d174e90bf7a9a94ad179ce




Accessing as speedyskill

Initialized MLflow to track repo "speedyskill/swiggy-delivery-time-prediction"

Repository speedyskill/swiggy-delivery-time-prediction initialized!

In [5]:
import mlflow
mlflow.set_tracking_uri('https://dagshub.com/speedyskill/swiggy-delivery-time-prediction.mlflow')

In [47]:
mlflow.set_experiment('Exp 8 - Final_Estimator')

2025/04/29 15:16:05 INFO mlflow.tracking.fluent: Experiment with name 'Exp 8 - Final_Estimator' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/10ca51f2f93b40bebcaa41c38accc71b', creation_time=1745939765311, experiment_id='8', last_update_time=1745939765311, lifecycle_stage='active', name='Exp 8 - Final_Estimator', tags={}>

In [8]:
pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 5.7 MB/s eta 0:00:00


In [9]:
df=pd.read_csv('cleaned_data.csv')
df.head()

,rider_id,age,ratings,restaurant_latitude,restaurant_longitude,delivery_latitude,delivery_longitude,order_date,weather,traffic,...,city_name,order_day,order_month,order_day_of_week,is_weekend,order_time_hour,order_time_of_day,pickup_time,distance,distance_type
0,INDORES13DEL02,37.0,4.9,22.745049,75.892471,22.765049,75.912471,2022-03-19,sunny,high,...,INDO,19,3,saturday,1,11.0,morning,15.0,3.025149,short
1,BANGRES18DEL02,34.0,4.5,12.913041,77.683237,13.043041,77.813237,2022-03-25,stormy,jam,...,BANG,25,3,friday,0,19.0,evening,5.0,20.183530,very_long
2,BANGRES19DEL01,23.0,4.4,12.914264,77.678400,12.924264,77.688400,2022-03-19,sandstorms,low,...,BANG,19,3,saturday,1,8.0,morning,15.0,1.552758,short
3,COIMBRES13DEL02,38.0,4.7,11.003669,76.976494,11.053669,77.026494,2022-04-05,sunny,medium,...,COIMB,5,4,tuesday,0,18.0,evening,10.0,7.790401,medium
4,CHENRES12DEL01,32.0,4.6,12.972793,80.249982,13.012793,80.289982,2022-03-26,cloudy,high,...,CHEN,26,3,saturday,1,13.0,afternoon,15.0,6.210138,medium


In [10]:
# drop columns not required for model input

columns_to_drop =  ['rider_id',
                    'restaurant_latitude',
                    'restaurant_longitude',
                    'delivery_latitude',
                    'delivery_longitude',
                    'order_date',
                    "order_time_hour",
                    "order_day",
                    "city_name",
                    "order_day_of_week",
                    "order_month"]

df.drop(columns=columns_to_drop, inplace=True)

df

,age,ratings,weather,traffic,vehicle_condition,type_of_order,type_of_vehicle,multiple_deliveries,festival,city_type,time_taken,is_weekend,order_time_of_day,pickup_time,distance,distance_type
0,37.0,4.9,sunny,high,2,snack,motorcycle,0.0,no,urban,24,1,morning,15.0,3.025149,short
1,34.0,4.5,stormy,jam,2,snack,scooter,1.0,no,metropolitian,33,0,evening,5.0,20.183530,very_long
2,23.0,4.4,sandstorms,low,0,drinks,motorcycle,1.0,no,urban,26,1,morning,15.0,1.552758,short
3,38.0,4.7,sunny,medium,0,buffet,motorcycle,1.0,no,metropolitian,21,0,evening,10.0,7.790401,medium
4,32.0,4.6,cloudy,high,1,snack,scooter,1.0,no,metropolitian,30,1,afternoon,15.0,6.210138,medium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45497,30.0,4.8,windy,high,1,meal,motorcycle,0.0,no,metropolitian,32,0,morning,10.0,1.489846,short
45498,21.0,4.6,windy,jam,0,buffet,motorcycle,1.0,no,metropolitian,36,0,evening,15.0,NaN,NaN
45499,30.0,4.9,cloudy,low,1,drinks,scooter,0.0,no,metropolitian,16,0,night,15.0,4.657195,short
45500,20.0,4.7,cloudy,high,0,snack,motorcycle,1.0,no,metropolitian,26,0,afternoon,5.0,6.232393,medium


In [11]:
temp_df=df.dropna().copy()

In [12]:
X=temp_df.drop(columns='time_taken')
y=temp_df['time_taken']

In [48]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [49]:
num_cols = ["age","ratings","pickup_time","distance"]

nominal_cat_cols = ['weather',
                    'type_of_order',
                    'type_of_vehicle',
                    "festival",
                    "city_type",
                    "is_weekend",
                    "order_time_of_day"]

ordinal_cat_cols = ["traffic","distance_type"]

In [52]:
# generate order for ordinal encoding

traffic_order = ["low","medium","high","jam"]

distance_type_order = ["short","medium","long","very_long"]

In [51]:
preprocessor=ColumnTransformer(transformers=[
    ('scale',MinMaxScaler(),num_cols),
    ('nominal_encode',OneHotEncoder(drop='first',handle_unknown='ignore',sparse_output=False),nominal_cat_cols),
    ('ordinal_encode',OrdinalEncoder(categories=[traffic_order,distance_type_order]),ordinal_cat_cols)
],remainder='passthrough',force_int_remainder_cols=False,n_jobs=-1,verbose_feature_names_out=False)

preprocessor.set_output(transform='pandas')

ColumnTransformer(force_int_remainder_cols=False, n_jobs=-1,
                  remainder='passthrough',
                  transformers=[('scale', MinMaxScaler(),
                                 ['age', 'ratings', 'pickup_time', 'distance']),
                                ('nominal_encode',
                                 OneHotEncoder(drop='first',
                                               handle_unknown='ignore',
                                               sparse_output=False),
                                 ['weather', 'type_of_order', 'type_of_vehicle',
                                  'festival', 'city_type', 'is_weekend',
                                  'order_time_of_day']),
                                ('ordinal_encode',
                                 OrdinalEncoder(categories=[['low', 'medium',
                                                             'high', 'jam'],
                                                            ['short', 'medium',
                                                             'long',
                                                             'very_long']]),
                                 ['traffic', 'distance_type'])],
                  verbose_feature_names_out=False)

In [53]:
X_train_trans=preprocessor.fit_transform(X_train)
X_test_trans=preprocessor.transform(X_test)

In [54]:
pt=PowerTransformer(method='yeo-johnson')
y_train_pt=pt.fit_transform(y_train.values.reshape(-1,1))
y_test_pt=pt.transform(y_test.values.reshape(-1,1))

In [55]:
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error,r2_score
from sklearn.model_selection import cross_val_score
from sklearn.compose import TransformedTargetRegressor
from sklearn.ensemble import StackingRegressor

In [28]:
import optuna

In [56]:
# build the best model

best_lgbm_params= {
    'n_estimators' : 105,
    'max_depth' : 39,
    'learning_rate' : 0.23688131888261493,
    'subsample' : 0.5173176053180517,
    'min_child_weight' : 12,
    'min_split_gain' : 0.0007351518634197696,
    'reg_lambda' : 1.012414361562786
}



best_rf_params= {
    'n_estimators' : 342,
    'max_depth' : 15,
    'max_features' : None,
    'min_samples_split' : 7,
    'min_samples_leaf' : 6,
    'max_samples' : 0.70279720384227
}


best_lgbm=LGBMRegressor(**best_lgbm_params)
best_rf=RandomForestRegressor(**best_rf_params)

lin_reg=LinearRegression()

In [57]:
# build the stacking regressor

stacking_regressor=StackingRegressor(
    estimators=[('best_lgbm',best_lgbm),('best_rf',best_rf)],
    final_estimator=lin_reg,
    cv=5,n_jobs=-1
)


model=TransformedTargetRegressor(regressor=stacking_regressor,transformer=pt)

model.fit(X_train_trans,y_train)


TransformedTargetRegressor(regressor=StackingRegressor(cv=5,
                                                       estimators=[('best_lgbm',
                                                                    LGBMRegressor(learning_rate=0.23688131888261493,
                                                                                  max_depth=39,
                                                                                  min_child_weight=12,
                                                                                  min_split_gain=0.0007351518634197696,
                                                                                  n_estimators=105,
                                                                                  reg_lambda=1.012414361562786,
                                                                                  subsample=0.5173176053180517)),
                                                                   ('best_rf',
                                                                    RandomForestRegressor(max_depth=15,
                                                                                          max_features=None,
                                                                                          max_samples=0.70279720384227,
                                                                                          min_samples_leaf=6,
                                                                                          min_samples_split=7,
                                                                                          n_estimators=342))],
                                                       final_estimator=LinearRegression(),
                                                       n_jobs=-1),
                           transformer=PowerTransformer())

In [58]:
# get the predictions
y_pred_train=model.predict(X_train_trans)
y_pred_test=model.predict(X_test_trans)

In [59]:
train_error=mean_absolute_error(y_train,y_pred_train)
test_error=mean_absolute_error(y_test,y_pred_test)


In [60]:
train_r2=r2_score(y_train,y_pred_train)
test_r2=r2_score(y_test,y_pred_test)

In [62]:
cv_scores=cross_val_score(model,
                          X_train_trans,y_train,
                          cv=3,scoring='neg_mean_absolute_error',
                          n_jobs=-1)

In [63]:
-cv_scores

array([3.05206132, 3.07594028, 3.04464107])

In [64]:
with mlflow.start_run():
  mlflow.set_tag('model','stacking_regressor')
  mlflow.log_params(stacking_regressor.get_params())
  mlflow.log_metric('train_error',train_error)
  mlflow.log_metric('test_error',test_error)
  mlflow.log_metric('train_r2',train_r2)
  mlflow.log_metric('test_r2',test_r2)
  mlflow.log_metric('cv_scores',-(cv_scores.mean()))

  # log the stacking regressor
  mlflow.sklearn.log_model(stacking_regressor,'model')

2025/04/29 15:40:31 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run gregarious-newt-758 at: https://dagshub.com/speedyskill/swiggy-delivery-time-prediction.mlflow/#/experiments/8/runs/0cd41a611a564a8cb28629fd5efba12d
🧪 View experiment at: https://dagshub.com/speedyskill/swiggy-delivery-time-prediction.mlflow/#/experiments/8
